In [1]:
import sys

sys.path.insert(0, "../..")
from pathlib import Path
from src.data import dataloader
from src.data import make_dataset

2022-06-13 18:38:39.968011: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-13 18:38:39.968076: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
eegPath = make_dataset.get_eeg()

2022-06-13 18:38:43.202 | INFO     | src.data.make_dataset:get_eeg:18 - Data is downloaded to ../../data/raw/datasets/eeg.


In [3]:
dataset = dataloader.BaseDataset(datapath = eegPath) #make dataset
print(len(dataset))

24


In [4]:
dataset.__getitem__(0)[0].shape #get shape of 1st item 

torch.Size([188, 14])

First the BaseDataStreamer is used (this is level one). This iterator automatically changes the window size if its larger than the smallest chunck in BaseDataSet. It also takes another chunck if the chunck does not have enough lines. For these reasons we always have the same shape tensors in one batch. The downside is that we throw away information by doing this.

In [9]:
loader = dataloader.BaseDataStreamer(dataset = dataset, window_size=10, batchsize=32)
x, y = loader.stream()
x.shape, y.shape

(torch.Size([32, 10, 14]), torch.Size([32]))

If trying to use a window size of 21, the BaseDataIterator will tell and set to 21:

In [10]:
loader = dataloader.BaseDataStreamer(dataset = dataset, window_size=25, batchsize=32)
x,y = loader.stream()

Maximum window length is 21, setting window length to 21.
                Use PaddedDataStreamer for bigger window size


In [11]:
x

tensor([[[4328.7202, 3967.1799, 4260.0000,  ..., 4302.0498, 4680.0000,
          4431.7900],
         [4335.3799, 3976.4099, 4264.1001,  ..., 4300.0000, 4681.0298,
          4429.2300],
         [4327.1802, 3971.7900, 4254.8701,  ..., 4296.4102, 4674.8701,
          4420.0000],
         ...,
         [4309.2300, 3964.1001, 4260.0000,  ..., 4289.7402, 4663.5898,
          4407.1802],
         [4298.4600, 3958.9700, 4250.2598,  ..., 4285.1299, 4663.0801,
          4406.1499],
         [4295.8999, 3958.9700, 4245.6401,  ..., 4290.7700, 4664.6201,
          4409.7402]],

        [[4302.5601, 3989.7400, 4266.1499,  ..., 4282.5601, 4608.7202,
          4366.6699],
         [4302.5601, 3986.6699, 4264.1001,  ..., 4284.1001, 4609.7402,
          4370.7700],
         [4297.4399, 3986.1499, 4264.1001,  ..., 4285.6401, 4606.1499,
          4367.1802],
         ...,
         [4303.5898, 3987.6899, 4263.5898,  ..., 4289.2300, 4615.8999,
          4360.5098],
         [4302.0498, 3985.6399, 4261.029

Another way of using the data is padding the data if the window size is too big (this is level 2). In the PaddedDataIterator the data is padded if the sequence is too short. For this reason we can use a larger window size than the size of the shortest observation. The padded data iterator also has an extra argument, min_nr_lines which specifies how many items there should at least be in an observation, if this condition does not equal to true, the previous window will be taken. 

In [12]:
loaderPadded = dataloader.PaddedDataStreamer(dataset = dataset, window_size=40, batchsize=32, min_nr_lines= 5)
x,y = loaderPadded.stream()
x,y = loaderPadded.stream()

x.shape, y.shape

(torch.Size([32, 40, 14]), torch.Size([32]))

In [13]:
x

tensor([[[4299.4902, 3995.8999, 4260.0000,  ..., 4281.0298, 4583.0801,
          4349.7402],
         [4300.0000, 3995.3799, 4266.1499,  ..., 4285.1299, 4587.1802,
          4351.7900],
         [4299.4902, 3993.8501, 4265.6401,  ..., 4286.6699, 4587.6899,
          4349.7402],
         ...,
         [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
             0.0000],
         [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
             0.0000],
         [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
             0.0000]],

        [[4312.3101, 4022.0500, 4278.4600,  ..., 4284.6201, 4612.3101,
          4368.2100],
         [4304.1001, 4016.9199, 4273.8501,  ..., 4282.5601, 4606.1499,
          4364.1001],
         [4303.0801, 4016.9199, 4270.7700,  ..., 4276.9199, 4602.0498,
          4362.5601],
         ...,
         [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
             0.0000],
         [   0.0000,    0.0000,    0.000